# 합성 테스트 데이터셋 생성

**왜 합성 테스트 데이터(Synthetic Test Dataset) 인가?**

RAG(검색 증강 생성) 증강 파이프라인의 성능을 평가하는 것은 매우 중요합니다. 

그러나 문서에서 수백 개의 QA(질문-문맥-응답) 샘플을 수동으로 생성하는 것은 시간과 노동력이 많이 소요될 수 있습니다. 또한 사람이 만든 질문은 철저한 평가에 필요한 복잡성 수준에 도달하기 어려워 궁극적으로 평가의 품질에 영향을 미칠 수 있습니다. 

합성 데이터 생성을 사용하면 데이터 집계 프로세스에서 **개발자의 시간을 90%** 까지 줄일 수 있습니다.

- RAGAS: https://docs.ragas.io/en/latest/concepts/testset_generation.html

아래의 주석을 해제한 후 실행하여 패키지를 설치 후 진행해주세요

**참고**

(2024.10.08 현재) ragas 패키지는 0.2.16 버전의 langchain 패키지와 호환됩니다.

따라서 버전을 0.2.16 버전으로 설치해야 합니다. (현재, 0.3.x 버전으로 우리가 사용하는 버전이 더 상위 버전입니다)

아래의 주석을 해제 후 진행해 주세요!

In [1]:
!pip install -q langchain==0.2.16
!pip install -q ragas==0.1.19

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.2.4 requires langchain-core>=0.3.60, but you have langchain-core 0.2.43 which is incompatible.
langchain-cohere 0.1.4 requires langchain-core<0.2.0,>=0.1.42, but you have langchain-core 0.2.43 which is incompatible.
langchain-community 0.3.27 requires langchain<1.0.0,>=0.3.26, but you have langchain 0.2.16 which is incompatible.
langchain-community 0.3.27 requires langchain-core<1.0.0,>=0.3.66, but you have langchain-core 0.2.43 which is incompatible.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have langchain-core 0.2.43 which is incompatible.
langchain-google-genai 2.1.6 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.
langchain-google-genai 2.1.6 requires langchain-core<0.4.0,>=0.3.66,

In [2]:
import langchain
import ragas

print(f"LangChain Version: {langchain.__version__}")
print(f"Ragas Version: {ragas.__version__}")

d:\git\langchain-kr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LangChain Version: 0.2.17
Ragas Version: 0.1.19


In [3]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


## 문서 전처리

문서를 로드 합니다.

In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)

19

각 문서 객체에는 `metadata` 를 통해 액세스할 수 있는 문서에 대한 추가 정보를 저장하는 데 사용할 수 있는 메타데이터 사전이 포함되어 있습니다. 

메타데이터 사전에는 `filename` 이라는 키가 포함되어 있는지 확인하세요. 

이 키는 Test datasets 생성 프로세스에서 활용될 것이므로. 메타데이터의 `filename` 속성은 동일한 문서에 속한 청크를 식별하는 데 사용됩니다. 

In [5]:
docs[0].metadata

{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 3,
 'total_pages': 23,
 'Author': 'dj',
 'Creator': 'Hwp 2018 10.0.0.13462',
 'Producer': 'Hancom PDF 1.3.0.542',
 'CreationDate': "D:20231208132838+09'00'",
 'ModDate': "D:20231208132838+09'00'",
 'PDFVersion': '1.4'}

In [6]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

## 데이터셋 생성

In [7]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o-mini")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o-mini")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

DocumentStore를 초기화합니다. 사용자 정의 LLM과 임베딩을 사용합니다.

In [8]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

TestSet 을 생성합니다.

In [9]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

**질문의 유형별 분포**

- simple: 간단한 질문
- reasoning: 추론이 필요한 질문
- multi_context: 여러 맥락을 고려해야 하는 질문
- conditional: 조건부 질문

In [10]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 0.4, reasoning: 0.2, multi_context: 0.2, conditional: 0.2}

- documents: 문서 데이터
- test_size: 생성할 질문의 수
- distributions: 질문 유형별 분포
- with_debugging_logs: 디버깅 로그 출력 여부

In [11]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs,
    test_size=10,
    distributions=distributions,
    with_debugging_logs=True,
    raise_exceptions=False,
)

Generating:   0%|          | 0/10 [00:00<?, ?it/s]              [ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 3, 'score': 2.0}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['CTA Conference', 'CES 2024', 'AIMLA 2024', 'AAAI Conference', 'Artificial Intelligence']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ["LLM 'Tongyi Qianwen 2.0'", 'AI model performance', 'MMLU benchmark', 'API capabilities', 'Generative AI advancements']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 3, 'score': 2.0}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['CTA Conference', 'CES 2024', 'AIMLA 2024', 'AAAI Conference', 'Artificial Intelligence']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'r

In [12]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = testset.to_pandas()
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the focus of AIMLA 2024 and when will ...,[Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(CT...,AIMLA 2024 focuses on artificial intelligence ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What architectural factors influence the accur...,[∙ 또한 크라우드소싱 플랫폼이 할당한 라이선스는 데이터셋 원저작자의 의도보다 더 ...,The answer to given question is not present in...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What significant event related to large langua...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n갈릴레오의 LL...,The publication 'Galileo' reported on November...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What is the purpose of the Executive Order on ...,[문제를 방지하는 조치를 확대\n∙ 형사사법 시스템에서 AI 사용 모범사례를 개발하...,The purpose of the Executive Order on Artifici...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What AI events linked to CES and AIMLA are hap...,[Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(CT...,The AI events linked to CES in 2024 include CE...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
5,Who's the new Exec Dir of the Frontier Model F...,[중점 지원할 예정\n∙ 포럼에 따르면 AI 레드팀에 대한 자금 지원은 AI 모델의...,The new Executive Director of the Frontier Mod...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
6,What role do governance frameworks play in AI ...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 과학혁신기...,The context does not provide specific informat...,multi_context,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
7,What changes in workflow and UI design come wi...,"[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n빌 게이츠, A...",The context does not provide specific details ...,multi_context,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
8,What are the risks of non-compliance with the ...,"[SPRi AI Brief |\n2023-12월호\nEU AI 법 3자 협상, 기반...",The answer to given question is not present in...,conditional,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
9,What ethical and operational priorities for AI...,[AD&D는 향후 5년간 카테고리 중 가장 높은 38.7%의 연평균 성장률이 예상됨...,The answer to given question is not present in...,conditional,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


DataFrame 에 저장된 데이터셋을 csv 파일로 저장합니다

In [13]:
# DataFrame의 상위 5개 행 출력
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the focus of AIMLA 2024 and when will ...,[Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(CT...,AIMLA 2024 focuses on artificial intelligence ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What architectural factors influence the accur...,[∙ 또한 크라우드소싱 플랫폼이 할당한 라이선스는 데이터셋 원저작자의 의도보다 더 ...,The answer to given question is not present in...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What significant event related to large langua...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n갈릴레오의 LL...,The publication 'Galileo' reported on November...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What is the purpose of the Executive Order on ...,[문제를 방지하는 조치를 확대\n∙ 형사사법 시스템에서 AI 사용 모범사례를 개발하...,The purpose of the Executive Order on Artifici...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What AI events linked to CES and AIMLA are hap...,[Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(CT...,The AI events linked to CES in 2024 include CE...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


In [14]:
# DataFrame을 CSV 파일로 저장
test_df.to_csv("data/ragas_synthetic_dataset.csv", index=False)